# Trabajo Práctico Final - MSSCAE 2024 - Grupo 5

---
## ***"Dinámica de precios a partir de la interacción entre agentes económicos"***

#### Integrantes:
* Cubino, Santiago
* Demare, Matías
* Puerta, Ezequiel

---
## Definición del sistema a estudiar

Un mercado con consumidores y productores de un mismo producto, donde los precios son definidos por cada productor individualmente, y los consumidores buscan siempre el menor precio en su vecindario (ver el próximo inciso: "Visión del consumidor").

Cada productor tiene asignado un costo fijo (renta) y un costo marginal (vinculado a una unidad de producto). Así, actualizan su capital de acuerdo con la cantidad de ventas y los precios asociados. Los productores luego varían los precios intentando maximizar su ganancia iteración a iteración (ver inciso "Dinámica de variación de precio").

### Suposiciones:
* El mercado se modela mediante una grilla (es un autómata celular con topología de toroide).
* La demanda es constante y los consumidores tienen capital ilimitado. Además, los productores tienen stock ilimitado
* El sistema evoluciona a pasos discretos, de manera sincrónica y en orden (primero los consumidores efectúan las compras y luego los productores actualizan sus precios)

### Suposiciones de los Productores
Comienzan con:
* Un determinado capital.
* Un costo fijo de operaciones.
* Un costo marginal por unidad de producto.
* Un período de tiempo (en cantidad de iteraciones) para analizar la ganancia obtenida.
* Un precio inicial (por encima del costo marginal según un *ratio* parametrizado).
* Los Productores pueden endeudarse y permitir que su capital sea negativo (bancarrota).
* Si se desea, el modelo permite eliminar a los Productores en bancarrota.
* El rendimiento obtenido en el período impacta directamente en el capital.

### Visión del consumidor

Dado que los agentes (productores y consumidores) comparten la grilla, y que por lo general son dispuestos al azar según una probabilidad para los productores, puede ocurrir que algunos consumidores queden muy alejados y aislados de cualquier productor. Si en el presente modelo utilizaramos alguno de los vecindarios clásicos de la literatura de autómatas celulares (Moore o Von Neumann), esta situación sería muy recurrente.

Por este motivo desarrollamos un vecindario de Moore "extendido" según un determinado nivel. Cada nivel se compone por las celdas del cuadrado que engloba las celdas del nivel anterior. El siguiente gráfico ilustra un vecindario de nivel 3.

![Vecindario de Moore Extendido](../img/extended_moore.png)

De esta manera, buscamos minimizar los casos de consumidores que queden aislados sin poder interactuar con los productores de la grilla.

### Dinámica de variación de precio
Cada productor tiene un costo marginal (costo de producir cada producto), y un costo fijo (se cobra en cada unidad de tiempo, independientemente de la cantidad de ventas, representa por ejemplo el alquiler y la luz).

Además, cada productor tiene una variable "dirección", que indica si el productor está aumentando o disminuyendo su precio (se inicializa aleatoriamente)

Los productores buscan en todo momento maximizar su ganancia, siguiendo una metodología "greedy". Para esto, evalúan su ganancia en cada intervalo de tiempo, usando la siguiente fórmula:
$$
G_t = (Precio_t - CostoMarginal) * Ventas_t - CostoFijo
$$

Luego, comparan $G_t$ con $G_{t-1}$:
- Si la ganancia aumentó (o se manuvo igual), se mantiene la dirección actual (si estaba aumentando el precio, sigo aumentando; y si estaba disminuyendo, sigo disminuyendo)
- Si la ganancia disminuyó, se invierte la dirección (si estaba aumentando el precio, paso a disminuir, y viceversa)

Además, hay un par de reglas extra que experimentalmente vimos que ayudan al modelo a producir mejores resultados:
- Si en un periodo no se tuvo ventas, disminuir el precio, ya que es seguro que el precio está demasiado alto (comparado con sus productores vecinos)
- El precio nunca puede ser menor al costo marginal, ya que no tiene sentido

La variación de precio (tanto aumento como disminución) es un porcentaje del precio actual. Este porcentaje es el mismo para todos los productores, un 2%

Así, la funcion que computa el nuevo precio queda:
```python
    def __compute_new_price(self) -> float:
        # self.__current_factor es la variable que representa la dirección actual
        # self.delta_price es el porcentaje de variación de precios (siempre 2%)
        
        current_profit = self._apply(sales=self.sales_within_period)
        self.previous_profit = self.last_profit
        self.last_profit = current_profit
        increased = current_profit >= self.previous_profit

        if self.sales_within_period == 0:
            # Si no vendí, bajar precios
            self.__current_factor = -1
        elif not increased or self.price == self.marginal_cost:
            # Si mi ganancia bajó, o llegué al costo marginal, invierto la dirección
            self.__current_factor = self.__current_factor * (-1)

        delta = self.delta_price * self.__current_factor
        return max(self.marginal_cost, self.price * (1 + delta))
```

---
## Introducción

In [ ]:
# Inicializacion

import sys
if '../' not in sys.path:
    sys.path.append('../')

import random
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from simulab.simulation.core.runner import Runner
from simulab.simulation.core.neighborhood import Moore, ExpandedMoore
from simulab.simulation.core.experiment import ExperimentParametersSet
from simulab.simulation.core.equilibrium_criterion import WithoutCriterion

from src.market import Market

from simulab.simulation.plotters.final_grid import FinalGridSeries
from simulab.simulation.plotters.numerical_series import NumericalSeries
from simulab.simulation.plotters.categorical_animated_lattice import (
    CategoricalAnimatedLatticeSeries,
)

---
### Ejemplo básico

Para implementar nuestro modelo hemos utilizado un pequeño *framework* construido durante la cursada de la presente materia. El repositorio del mismo está en GitHub y PyPi bajo el nombre de [SimuLab](https://github.com/EzequielPuerta/simulab).

En el siguiente bloque de código podremos observar una simulación básica utilizando dicho *framework*, para ilustrar como utilizar el modelo desarrollado.

Podemos agrupar los parámetros de nuestro modelo en la clase `ExperimentParametersSet`. La misma acepta una lista de valores para cada uno de dichos parámetros. De esta manera, se pueden barrer varios valores de un mismo parámetro.

Tenemos la posibilidad de informar la cantidad máxima de iteraciones a realizar mediante `max_steps` y de utilizar un criterio para detener la simulación antes de alcanzar el máximo de iteraciones (al encontrar un equilibrio en algún valor observable de la simulación por ejemplo). En nuestro caso, para mantenerlo simple, no utilizaremos criterio de parada (por eso pasamos una instancia de `WithoutCriterion`) y dejamos la cantidad máxima de iteraciones en 500.

Todos esos datos se los pasamos al `Runner` junto al modelo a simular, en nuestro caso el llamado `Market`. `Runner` contiene toda la lógica necesaria para poder simular nuestro modelo con los parámetros deseados. Para iniciar la simulación, debemos ejecutar el método `start()` del `runner`.

In [ ]:
from scenarios.basic_example import config as basic_example_conf

random.seed(2000)
np.random.seed(2000)

experiment_parameters_set = ExperimentParametersSet(
    length=[20],
    neighborhood=[ExpandedMoore(3)],
    agent_types=[2],
    capital=[2500],
    producer_probability=[0.15],
    profit_period=[7],
    price_ratio=[(1.2, 1.5)],
    fixed_cost=[(10, 1)],
    marginal_cost=[(10, 1)],
    quantity_to_buy=[(1, 0)],
    bankrupt_enabled=[False],
    configuration=[basic_example_conf]
)
criterion = WithoutCriterion()
runner0 = Runner(Market, experiment_parameters_set, criterion, max_steps=500)
runner0.start()

El runner se encargará no solo de simular el modelo, sino también de ir calculando y almacenando una variedad de datos de gran utilidad para el posterior análisis del modelo. Estos datos a calcular son definidos en la clase del propio modelo, en nuestro caso en el archivo `src/market.py`, bajo los métodos decorados como `@as_series`.

Todo método de `Market` que tenga algún decorador que lo identifique como una serie de datos, será ejecutado en cada iteración y se almacenarán los datos producidos. Una vez terminada la simulación, podremos utilizar el `runner` para graficar dichos datos mediante los distintos `plotters` que contiene la libreria, o acceder a los resultados precalculados para crear nuevas gráficas mas acordes a las necesidades de la situación.

A continuación se muestra uno de los `plotters` disponibles, llamado `CategoricalAnimatedLatticeSeries`, que nos permite obtener una animación (basada en *Plotly*) del primer experimento del `runner` (por si se realizaron varias simulaciones en el mismo `runner`). La animación graficará la serie `price_categorized_lattice`, que es un snapshot de la grilla en cada iteración, haciendo foco solamente en los precios de cada agente. Al ser "categorized" se podrá filtrar por el tipo agente.

In [ ]:
CategoricalAnimatedLatticeSeries.show_up(
    "price_categorized_lattice",
    runner=runner0,
    experiment_id=0,
    plot_title="Evolución del precio",
    height=500,
    colorscale="Viridis",
    show_labels=False,
)

Luego, podemos observar la evolución del precio promedio

In [ ]:
NumericalSeries.show_up(
    "average_price",
    runner=runner0,
    plot_title="Precio promedio entre productores y consumidores",
    yaxis_title="Precio promedio",
)

Y la variación de precio y ganancia respecto al instante anterior

In [ ]:
NumericalSeries.show_up(
    "average_price_change",
    runner=runner0,
    plot_title="Incremento porcentual de precios (promedio entre productores)",
    yaxis_title="Incremento promedio",
)
NumericalSeries.show_up(
    "average_profit_change",
    runner=runner0,
    plot_title="Incremento porcentual de ganancia (promedio entre productores)",
    yaxis_title="Incremento promedio",
)

---
### Barrido del parámetro `producer_probability`

Dicho ya lo básico de nuestro modelo, pasemos a un ejemplo un poco mas interesante. Hagamos un barrido del parámetro `producer_probability`, pasando esta vez no una lista con un único valor, sino con todos los valores que deseamos analizar.

In [ ]:
experiment_parameters_set = ExperimentParametersSet(
    length=[10],
    neighborhood=[ExpandedMoore(3)],
    agent_types=[2],
    capital=[2500],
    producer_probability=[0.05, 0.10, 0.15, 0.50],
    profit_period=[7],
    price_ratio=[(1.2, 1.5)],
    fixed_cost=[(10, 1)],
    marginal_cost=[(10, 1)],
    quantity_to_buy=[(1, 0)],
    bankrupt_enabled=[False],
)
criterion = WithoutCriterion()
runner1 = Runner(Market, experiment_parameters_set, criterion, max_steps=500)
runner1.start()

Y utilicemos otro `plotter`, llamado `FinalGridSeries`, que nos permitirá crear un gráfico compuesto por varios subplots, organizados en N filas y 2 columnas. La cantidad de filas es equivalente a la cantidad de experimentos simulados en el `runner`, que a su vez es igual al producto cartesiano de las listas provistas como parámetro en el `ExperimentParametersSet`.

En nuestro caso, todos los parámetros tienen listas de un único elemento salvo por el mencionado atributo `producer_probability`, que tiene una lista con 4 valores distintos. Por lo tanto N=4.

Volviendo al gráfico, como dijimos, cada fila representa un experimento con su combinación de parámetros. Por otro lado, cada columna representa un instante de la simulación. La primera columna es el instante inicial. La segunda columna es la última iteración de cada simulación. No necesariamente tienen que ser en la misma iteración.

In [ ]:
FinalGridSeries.show_up(
    "price_lattice", 
    runner=runner1,
    plot_title=("Evolución de Precios de Mercado"),
    leyend="Precio",
    colorscale="Viridis",
)

Se observa que para las dos primeras simulaciones, con valores bajos de `producer_probability`, los precios se elevan, aparentemente sin techo. Por el contrario, para valores altos de `producer_probability`, los precios tienden a equilibrarse en toda la grilla alrededor del valor 10. No tan casualmente, es el mismo valor que indicamos para la media del costo marginal para los productores. Pero hablaremos de esto mas adelante.

El gráfico nos da la pauta que hay un momento en el cuál el valor de `producer_probability` (y por consiguiente, la cantidad de productores disponibles en la grilla) condiciona que ocurrirá con los precios del modelo.

Sin entrar en demasiado detalle porque ocurre ésto, hagamos un barrido mucho mas fino para detectar el valor de dicho punto crítico. Ésta vez, con valores entre 3% y 25%. Además, repitamos 30 veces la simulación para cada valor de probabilidad. De esa manera tendremos una muestra de precios bastante grande, categorizados según su `producer_probability`, que graficaremos con un Box Plot para estudiar la media y la dispersión en cada caso.

In [ ]:
# Barrido entre 0.03 y 0.25

criterion2 = WithoutCriterion()
prices, probabilities = [], []
_probabilities = [value/100 for value in range(3, 25)]
repetitions = 30
length = 10

for probability in _probabilities:
    for _ in range(repetitions):
        experiment_parameters_set2 = ExperimentParametersSet(
            length=[length],
            neighborhood=[ExpandedMoore(3)],
            agent_types=[2],
            capital=[5000],
            producer_probability=[probability],
            profit_period=[7],
            price_ratio=[(1.2, 1.5)],
            fixed_cost=[(10, 1)],
            marginal_cost=[(10, 1)],
            quantity_to_buy=[(1, 0)],
            bankrupt_enabled=[False],
        )
        runner2 = Runner(Market, experiment_parameters_set2, criterion2, max_steps=500)
        runner2.start()
        final_prices = sum(runner2.experiments[0].series["price_lattice"][-1], [])
        
        prices = prices + final_prices
        probabilities = probabilities + ([probability] * length**2)

prices_dataframe = pd.DataFrame({"price": prices, "probability": probabilities})
fig = px.box(prices_dataframe, x="probability", y="price")
fig.show()

Se observa que en general, el sistema tiene un comportamiento muy distinto para valores inferiores al 15% de probabilidad de productores, comparado con valores superiores. Analizaremos ahora con mas detalle que ocurre en situaciones alrededor de ese valor, con valores inferiores al mismo y también por supuesto, valores superiores.

---
## Escenario 1: Monopolio

En esta sección, queremos explorar el efecto que tienen los monopolios sobre el precio de un producto. La idea es observar cómo, en un mercado con poca competencia, los productores son libres de aumentar el precio sin límite (siempre bajo la hipótesis de consumidores con capital ilimitado y demanda constante).

### Monopolio absoluto

Para empezar, proponemos una configuración básica, en la que cada consumidor tiene un único productor al que le puede comprar. Notar que en este caso, al no haber consumidores compartidos, no hay realmente una señal de precio que compartan los distintos productores. Por lo tanto, no se trata realmente de un sistema complejo, ya que no hay interacciones no lineales entre agentes. Sin embargo, nos pareció un buen punto para comenzar, ya que permite observar comportamientos interesantes que ocurren en el caso límite.

In [ ]:
from scenarios.monopolios_basic import config as monopolios_basic_conf
np.random.seed(seed=12345)
random.seed(1234)

experiment_parameters_set = ExperimentParametersSet(
    length=[6],
    neighborhood=[Moore],
    agent_types=[2],
    producer_probability=[0.15],
    profit_period=[2],
    configuration = [monopolios_basic_conf],
    price_ratio=[(1.2, 5)],
    fixed_cost=[(20, 0)],
    marginal_cost=[(10, 1)],
    quantity_to_buy=[(1,0)],
    bankrupt_enabled=[False],
)
criterion = WithoutCriterion()
runner_monopolio = Runner(Market, experiment_parameters_set, criterion, max_steps=500)
runner_monopolio.start()

En la siguiente celda, se puede ver la distribución de agentes, donde los amarillos son los productores y los violetas son los consumidores.

En este caso, consideramos un vecindario de Moore tradicional

In [ ]:
CategoricalAnimatedLatticeSeries.show_up(
    "agent_types_categorized_lattice",
    runner=runner_monopolio,
    experiment_id=0,
    plot_title="Tipos de agentes",
    height=400,
)

Al ejecutarlo, podemos ver que rápidamente los productores se dan cuenta de que pueden subir los precios constantemente, aumentando así sus ganancias

In [ ]:
CategoricalAnimatedLatticeSeries.show_up(
    "price_categorized_lattice",
    runner=runner_monopolio,
    experiment_id=0,
    plot_title="Evolución del precio",
    height=500,
    zmin=0,
    zmax=300,
)

Se puede ver además que tanto las ganancias de los productores, como los precios de los productos siguen un crecimiento exponencial (con exponente ~0.02, que es el incremento de precio por paso configurado por default)

In [ ]:
productor = (1, 1)
productor_profit_series = [iter[productor[0]][productor[1]]
                           for iter in runner_monopolio.experiments[0].series['profit_categorized_lattice']]
runner_monopolio.experiments[0].series['productor_profit_series'] = [profit for profit, agent_type in productor_profit_series]

NumericalSeries.show_up(
    "productor_profit_series",
    runner=runner_monopolio,
    plot_title="Ganancia de un productor",
    yaxis_title="Ganancia",
)

NumericalSeries.show_up(
    "average_price",
    runner=runner_monopolio,
    plot_title="Precio promedio entre productores y consumidores",
    yaxis_title="Precio promedio",
)

Además podemos ver que, luego de un pequeño intervalo en el que prueban hacia donde mover los precios, el incremento porcentual de precios (respecto a t-1) es del 2% (otra vez, indicando que todos los productores están incrementando su precio por el porcentaje definido)

In [ ]:
NumericalSeries.show_up(
    "average_price_change",
    runner=runner_monopolio,
    plot_title="Incremento porcentual de precios (promedio entre productores)",
    yaxis_title="Incremento promedio",
)

Además, el incremento porcentual de ganancia también tiende al 2%.

De hecho, es mayor al 2%, porque el costo marginal del producto cada vez se hace más insignificante, a medida que aumenta el precio del mismo.

In [ ]:
NumericalSeries.show_up(
    "average_profit_change",
    runner=runner_monopolio,
    plot_title="Incremento porcentual de ganancia (promedio entre productores)",
    yaxis_title="Incremento promedio",
)

### Aumento de precios en cadena

Luego, buscamos otra configuración que presente comportamientos de monopolios (es decir, que haya productores que tienen exclusividad sobre ciertos clientes), pero donde también existan otros consumidores compartidos entre los productores, para permitir la emergencia de comportamientos complejos.

Para eso, generamos una configuración aleatoria, con un 6% de productores, y un 94% de consumidores. Consideramos un vecindario "Expanded Moore (3)"

In [ ]:
from scenarios.monopolios_complex import config as monopolios_complex_conf

experiment_parameters_set = ExperimentParametersSet(
    length=[20],
    neighborhood=[ExpandedMoore(3)],
    agent_types=[2],
    configuration = [monopolios_complex_conf],
    producer_probability=[0.06],
    profit_period=[2],
    price_ratio=[(1.2, 1.5)],
    fixed_cost=[(20, 0)],
    marginal_cost=[(10, 1)],
    quantity_to_buy=[(1,0)],
    bankrupt_enabled=[False],
)
criterion = WithoutCriterion()
runner_cadena = Runner(Market, experiment_parameters_set, criterion, max_steps=500)
runner_cadena.start()

Como se puede ver, los productores que tienen monopolios sobre ciertas regiones suelen ser los primeros en subir el precio, ya que tienen al menos algunos compradores garantizados.

A su vez, los precios altos permiten que otros productores cercanos eleven sus precios sin perder ventas, por lo que también lo hacen, y esto se propaga hacia el resto de regiones, provocando que finalmente todos los precios se eleven.

In [ ]:
CategoricalAnimatedLatticeSeries.show_up(
    "agent_types_categorized_lattice",
    runner=runner_cadena,
    experiment_id=0,
    plot_title="Tipos de agentes de Mercado",
    height=400,
)
CategoricalAnimatedLatticeSeries.show_up(
    "price_categorized_lattice",
    runner=runner_cadena,
    experiment_id=0,
    plot_title="Evolución del precio en modelo de Mercado",
    height=500,
    zmin=0,
    zmax=300,
)

Otra vez, en presencia de tantos monopolios, el crecimiento de precios tiende a una exponencial.

In [ ]:
NumericalSeries.show_up(
    "average_price",
    runner=runner_cadena,
    plot_title="Precio promedio entre productores y consumidores",
    yaxis_title="Precio promedio",
)

Además, luego de un breve periodo de tiempo (en el que todavía hay competencia entre algunos productores cercanos), otra vez vemos que todos los proveedores incrementan el precio lo máximo posible en cada paso.

In [ ]:
NumericalSeries.show_up(
    "average_price_change",
    runner=runner_cadena,
    plot_title="Incremento porcentual de precios (promedio entre productores)",
    yaxis_title="Incremento promedio",
)

El incremento de ganancia parece tener un poco más de varianza al principio, pero después vuelve a estabilizarse en 2%.

In [ ]:
NumericalSeries.show_up(
    "average_profit_change",
    runner=runner_cadena,
    plot_title="Incremento porcentual de ganancia (promedio entre productores)",
    yaxis_title="Incremento promedio",
)

---
## Escenario 2: Equilibrio dinámico

En esta sección queremos estudiar el caso donde los productores estan distribuidos uniformemente, de forma tal que todo consumidor tenga a por lo menos 2 productores en su rango de visión.

In [ ]:
np.random.seed(seed=6)
random.seed(6)

In [ ]:
from scenarios.equilibrio_dinamico import config as equilibrio_dinamico_conf

experiment_parameters_set = ExperimentParametersSet(
    length=[20],
    neighborhood=[ExpandedMoore(2)],
    agent_types=[2],
    capital=[100],
    producer_probability=[0.24],
    profit_period=[2],
    price_ratio=[(1.3, 1.5)],
    fixed_cost=[(7, 0)],
    marginal_cost=[(10, 1)],
    quantity_to_buy=[(1,0)],
    bankrupt_enabled=[False],
    configuration=[equilibrio_dinamico_conf]
)
criterion = WithoutCriterion()
runner_equilibrio = Runner(Market, experiment_parameters_set, criterion, max_steps=200)
runner_equilibrio.start()

In [ ]:
FinalGridSeries.show_up(
    "price_lattice", 
    runner=runner_equilibrio,
    plot_title=("Evolución de Precios en un escenario competitivo"),
    leyend="Precio",
    height=400,
    colorscale="Viridis",
)

## Estabilización del precio

Podemos observar como ahora en un escenario más competitivo, los precios tienden a estavilizarse al rededor de un valor.

In [ ]:
CategoricalAnimatedLatticeSeries.show_up(
    "price_categorized_lattice",
    runner=runner_equilibrio,
    experiment_id=0,
    plot_title="Evolución del precio",
    height=500,
    colorscale="Viridis",
    show_labels=False,
    zmax=15,
    zmin=8,
)

In [ ]:
NumericalSeries.show_up(
    "average_consumer_price",
    runner=runner_equilibrio,
    plot_title="Precio promedio entre consumidores",
    yaxis_title="Precio promedio",
)

In [ ]:
NumericalSeries.show_up(
    "average_producer_price",
    runner=runner_equilibrio,
    plot_title="Precio promedio entre productores",
    yaxis_title="Precio promedio",
)

In [ ]:
NumericalSeries.show_up(
    "gini_prices_distribution",
    runner=runner_equilibrio,
    plot_title="Coeficiente de gini de los precios de los consumidores",
    yaxis_title="Coeficiente de gini",
)

Claramente, a nivel macroscopico, el precio tiende a estabilizarse cerca del costo marginal promedio de los productores. Abajo podemos observar la diferencia en las distribuciones de los precios de los productores al inicio y al final de la simulación:

In [ ]:
productores = set(runner_equilibrio.experiments[0]._by_type[1])
first = runner_equilibrio.experiments[0].series['price_lattice'][0]
last = runner_equilibrio.experiments[0].series['price_lattice'][-1]
first_data = [ [first[i][j], "t_0"] for (i, j) in productores ]
last_data = [ [last[i][j], "t_200"] for (i, j) in productores ]
data = first_data + last_data

fig = px.histogram(pd.DataFrame(data, columns=["Precio", "Iteración"]),
                   title="Histogramas de precios iniciales y finales para productores.",
                   x="Precio",
                   nbins=30,
                   color="Iteración")
fig.show()

## La ganancia de los productores

Por otro lado, pude resultarnos interesante estudiar que tan exitosos están siendo los productores. Esto lo hacemos viendo día a día, como varian las ganancias de cada uno.

In [ ]:
CategoricalAnimatedLatticeSeries.show_up(
    "profit_categorized_lattice",
    runner=runner_equilibrio,
    experiment_id=0,
    plot_title="Evolución de la ganancia entre los productores",
    height=500,
)

Nos encontramos con que tenemos productores que logran competir y mantener una ganancia positiva, pero también existen otros que están en un estado de frustración. Es decir, que no lograr hacer las ventas suficientes para superar el gasto fijo, debido a que no pueden ser competitivos en su vecindario.

In [ ]:
productor = (8, 7)
productor_profit_series = [iter[productor[0]][productor[1]]
                           for iter in runner_equilibrio.experiments[0].series['profit_categorized_lattice']]
runner_equilibrio.experiments[0].series['productor_profit_series'] = [profit for profit, agent_type in productor_profit_series]
NumericalSeries.show_up(
    "productor_profit_series",
    runner=runner_equilibrio,
    plot_title="Ganancia de un productor que logra competir",
    yaxis_title="Ganancia",
)

In [ ]:
productor = (9, 10)
productor_profit_series = [iter[productor[0]][productor[1]]
                           for iter in runner_equilibrio.experiments[0].series['profit_categorized_lattice']]
runner_equilibrio.experiments[0].series['productor_profit_series'] = [profit for profit, agent_type in productor_profit_series]
NumericalSeries.show_up(
    "productor_profit_series",
    runner=runner_equilibrio,
    plot_title="Ganancia de un productor frustrado",
    yaxis_title="Ganancia",
)

---
## Escenario 3: Bancarrota

Como acabamos de ver, actualmente si un productor termina con capital no positivo, puede seguir interactuando en la simulación, vendiendo sus productos y endeudándose sin límite.

Al no ser esto demasiado real, introducimos el concepto de *"bancarrota"*, que podrá ser activado con el atributo `bankrupt_enabled=True` del modelo `Market`.

Una vez activado, todo aquel productor cuyo capital no sea positivo, se lo considerará en bancarrota y no será contemplado en la simulación. Es decir, no podrá vender sus productos y si satisfacía a algún consumidor, éste deberá buscar otro productor a quién comprarle.

In [ ]:
from copy import deepcopy
from src.bankrupt_utils import *

random.seed(1000)
np.random.seed(1000)

In [ ]:
configuration = create_configuration()
experiment_parameters_set = ExperimentParametersSet(
    length=[20],
    neighborhood=[ExpandedMoore(3)],
    agent_types=[2],
    capital=[1_000],
    stock=[5_000_000_000],
    producer_probability=[0.20],
    profit_period=[7],
    price_ratio=[(1.2, 1.5)],
    fixed_cost=[(10, 1)],
    marginal_cost=[(10, 1)],
    quantity_to_buy=[(1, 0)],
    bankrupt_enabled=[False, True],
    configuration=[configuration],
)
criterion = WithoutCriterion()
runner_bankrupt_0 = Runner(Market, experiment_parameters_set, criterion, max_steps=1000)
runner_bankrupt_0.start()

A continuación analizaremos una configuración inicial bajo el concepto de bancarrota. Para comparar, también observemos la misma configuración pero con *bancarrota* desactivada.

Vemos como en ambos casos hay productores que quedan con montos de capital no positivos, pero en el primer caso, los productores siguen interactuando y afectando el precio de sus vecinos. Es decir, siguen compitiendo por la demanda disponible aún cuando no tienen capital para afrontar las deudas en las que incurrieron.

Por el contrario, con la bancarrota activada, observamos que varios productores desaparecen de la simulación, provocando que los productores escaseen y que algunos (beneficiados por la ubicación que obtuvieron al azar) tengan la posibilidad de incrementar sus precios dado que ven aumentar la demanda que reciben.

In [ ]:
FinalGridSeries.show_up(
    "capital_lattice", 
    runner=runner_bankrupt_0,
    plot_title="Evolución del capital de los productores",
    leyend="Capital",
    colorscale="Viridis",
)

In [ ]:
CategoricalAnimatedLatticeSeries.show_up(
    "profit_categorized_lattice",
    runner=runner_bankrupt_0,
    experiment_id=1,
    plot_title=f"Evolución de la ganancia de los productores",
    height=500,
)

Si observamos los precios en lugar del capital, vemos que sin bancarrota, los precios se estabilizan alrededor del costo marginal (como vimos antes), dado que la probabilidad de existencia de productores está por encima del punto crítico (en particular, para este caso parametrizamos %20).

Pero al activarse la bancarrota, la cantidad de productores desciende considerablemente con el paso de las iteraciones, por lo tanto disminuye la competencia y aumenta la probabilidad de encontrar regiones monopolizadas por los productores restantes que terminan elevando los precios al no encontrar resistencia de otros productores.

In [ ]:
FinalGridSeries.show_up(
    "price_lattice", 
    runner=runner_bankrupt_0,
    plot_title="Evolución de los precios",
    leyend="Precio",
    colorscale="Viridis",
)

Algo que podemos preguntarnos es si al activar la bancarrota (y por consiguiente, aumentar las chances de generar monopolios donde no había ninguno), estamos incrementando la cantidad de productores que caen en bancarrota con respecto a los que hubieran quedado con capital no positivo sin bancarrota activada.

Para ello vamos a hacer un barrido de probabilidades un poco mas fino, y dada cada probabilidad, simular 10 veces el modelo en ambas versiones: con y sin bancarrota. Todos los resultados los vamos a guardar en un diccionario que luego mostraremos en un gráfico de barras.

In [ ]:
repetitions = 10
max_steps = 1000
_probabilities = [value/100 for value in range(3, 30, 3)]

data = {}
data["Capital"] = [] 
data["Cantidad"] = []
data["Estrategia"] = []

for probability in _probabilities:
    for _ in range(repetitions):
        configuration = create_configuration(producer_probability=probability)
        execute_with(
            configuration,
            data,
            f"Sin bancarrota (prob: {probability})",
            max_steps=max_steps,
            bankrupt_enabled=False)
        execute_with(
            configuration,
            data,
            f"Con bancarrota (prob: {probability})",
            max_steps=max_steps,
            bankrupt_enabled=True)

Finalmente se observa que no hay demasiada diferencia de cantidad de productores entre las versiones con y sin bancarrota. Es más, en todos los casos hay una leve ventaja de los productores con capital positivo en la versión con bancarrota. O lo que es igual, en la versión con bancarrota, hay levemente menos productores que se quedan sin capital.

Podemos intuir que esto se debe a que en la versión con bancarrota, los productores sin capital dejan de interactuar, liberando una parte de la demanda para que busque nuevos productores a quienes comprar. Esto puede hacer que algunos productores que estaban cerca de la bancarrota, la eviten, recibiendo un afluente de demanda con la que no estaban contando previamente.

Por el contrario, en la versión sin bancarrota, los productores con capital negativo siguen interactuando y no liberan la demanda, por lo que pueden estar colaborando indirectamente en la pérdida de ganancias de otros productores, a tal punto de llevarlos también a tener capital negativo.

In [ ]:
data = pd.DataFrame(pd.DataFrame(data), columns=["Capital", "Cantidad", "Estrategia"])
fig = px.bar(
    data,
    title="Productores según su capital y estrategia de simulación",
    x="Estrategia",
    y="Cantidad",
    color="Capital",
    color_discrete_sequence=["tomato", "grey"],
)
fig.show()

---
## Trabajos futuros

#### Stock
* Los Productores actualmente poseen un atributo *stock* que decrementamos con las ventas pero que no estamos incrementando de ningún modo. Dado que el modelo permite que un Productor se endeude y siga vendiendo aún teniendo capital negativo (a menos que se active el *flag* de *"bancarrota"*), podría ser interesante que el Productor decida en que momentos permitir pérdidas o no, cuando invertir capital para recuperar stock, etc.

#### Topología
* Actualmente estamos usando una grilla única para ambos tipos de agentes, lo que nos llevó a utilizar el vecindario de Moore Extendido y tomar varias decisiones que no teníamos originalmente (Consumidores que quedan aislados sin poder acceder a Productor alguno, Productores que poseen distintas cantidades de Consumidores, etc). Esto se podría homogeneizar con una topología distinta, un modelo de doble grilla, una para cada tipo de agente, y que los mismos puedan interactuar con los 8 de enfrente (Moore clásico).

![Grilla doble](../img/grilla_doble.png)

#### Colusion
* Para romper la tendencia que el precio encuentre el equilibrio en el valor del costo marginal, se podría implementar algún tipo de colusión entre algunos agentes (por ejemplo, que productores cercanos puedan comunicarse, y así coordinar un precio de venta superior al costo marginal).